In [25]:
import pandas as pd
import ta
import pandas_ta
#from "../Miscelaneous/plot_utils" import plot_results

In [26]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [27]:
symbols = ['ETH']



for symbol in symbols:
    df[f'{symbol}_EMA_50'] = ta.trend.ema_indicator(df[f'{symbol}-USD_Close'], 50, fillna=False)
    
    df[f'{symbol}_EMA_200'] = ta.trend.ema_indicator(df[f'{symbol}-USD_Close'], 200, fillna=False)
    
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)
    print(pandas_ta.supertrend(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], period=10, length=None, multiplier=3))

    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'])

    dfs = pandas_ta.supertrend(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], period=10, length=None, multiplier=3)
    df[f'{symbol}_SuperDir'] = dfs['SUPERTd_7_3.0']

df.dropna(inplace=True)

       SUPERT_7_3.0  SUPERTd_7_3.0  SUPERTl_7_3.0  SUPERTs_7_3.0
0          0.000000              1            NaN            NaN
1               NaN              1            NaN            NaN
2               NaN              1            NaN            NaN
3               NaN              1            NaN            NaN
4               NaN              1            NaN            NaN
...             ...            ...            ...            ...
82034   3271.195453              1    3271.195453            NaN
82035   3271.195453              1    3271.195453            NaN
82036   3273.395231              1    3273.395231            NaN
82037   3276.941626              1    3276.941626            NaN
82038   3276.941626              1    3276.941626            NaN

[82039 rows x 4 columns]


In [28]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.balance_amount = balance_amount
        self.short_amount = 0
        self.long_amount = 0

        self.balance_unit = balance_unit
        self.short_unit = None
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = 0.9994      # VIP level 0, paying fees with BNB 0.99925
        self.symbols = symbols

        self.long_price = 0
        self.short_price = 0
        self.sell_price = 0

        self.take_profit = 3
        self.stop_loss = 2

        self.wins = 0
        self.losses = 0
        self.up_status = {}
        self.up_candlesticks = {}
        self.reset_up_status([symbols])
        self.reset_up_candlesticks([symbols])

        self.down_status = {}
        self.down_candlesticks = {}
        self.reset_down_status([symbols])
        self.reset_down_candlesticks([symbols])
    

    def buy_long(self, symbol, long_price, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price])
        self.long_price = long_price

    def buy_short(self, symbol, short_price, time):
        self.short_amount = (self.balance_amount / short_price) * self.trading_fee_multiplier
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price])
        self.short_price = short_price


    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'USDT'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = self.balance_amount + (self.short_amount * (self.short_price - sell_price) * self.trading_fee_multiplier)
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0



    def reset_up_status(self, symbols):
        for symbol in self.symbols:
            self.up_status[symbol] = 'None'
    
    def reset_up_candlesticks(self, symbols):
        for symbol in self.symbols:
            self.up_candlesticks[symbol] = 0


    def reset_down_status(self, symbols):
        for symbol in self.symbols:
            self.down_status[symbol] = 'None'
    
    def reset_down_candlesticks(self, symbols):
        for symbol in self.symbols:
            self.down_candlesticks[symbol] = 0


In [29]:
'''
Wait for 50 EMA to cross over 200 EMA

Wait for candlestick to hit 50 EMA
    Wait for until we break back over/under 50 EMA
    Make sure it took less than 3 candles to get back to the 50 EMA
    Check if Supertrend is in the right direction


Set stop-loss 2x ATR
Set take-profit at 3x ATR
'''

'\nWait for 50 EMA to cross over 200 EMA\n\nWait for candlestick to hit 50 EMA\n    Wait for until we break back over/under 50 EMA\n    Make sure it took less than 3 candles to get back to the 50 EMA\n    Check if Supertrend is in the right direction\n\n\nSet stop-loss 2x ATR\nSet take-profit at 3x ATR\n'

In [30]:
env = TradingEnv(100, 'USDT', symbols)
env.up_candlesticks

{'ETH': 0}

In [31]:
for i in range(len(df)):

    if i == 0:
        continue
    print(env.balance_amount)
    if env.balance_unit == 'USDT' and env.short_amount == 0:  # Want to buy
        for symbol in symbols:

            # If up_status is buy check if buy is available else add 1 to env.up_candlesticks
            if env.up_status[symbol] == 'Buy':
                if df[f'{symbol}-USD_Close'].iloc[i] >= df[f'{symbol}_EMA_50'].iloc[i] and df[f'{symbol}_SuperDir'].iloc[i] != 1: # Good to buy
                    print(f'{env.balance_amount} long', env.balance_unit, env.wins, env.losses)
                    env.buy_long(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    print(f'{symbol} long', df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    env.reset_up_candlesticks([symbol])
                    env.reset_up_status([symbol])
                else: # Not ready to buy
                    env.up_candlesticks[symbol] += 1
                    if env.up_candlesticks[symbol] == 3:
                        env.reset_up_candlesticks([symbol])
                        env.reset_up_status([symbol])
                        env.reset_down_candlesticks([symbol])
                        env.reset_down_status([symbol])
                        

            # Check if up_cross
            if df[f'{symbol}_EMA_50'].iloc[i-1] <= df[f'{symbol}_EMA_200'].iloc[i-1] and df[f'{symbol}_EMA_50'].iloc[i] > df[f'{symbol}_EMA_200'].iloc[i]:
                env.up_status[symbol] = 'Crossed'
                print(f'{symbol} Up Crossed', df['CloseTime'].iloc[i])
            
            # Checl if up_cross and thouched
            if env.up_status[symbol] == 'Crossed' and df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_EMA_50'].iloc[i]:
                env.up_status[symbol] = 'Buy'
                print(f'{symbol} buy signal', df['CloseTime'].iloc[i])


            ###################################################################################################################################################
            # If down_status is sell check if sell is available else add 1 to env.up_candlesticks

            if env.down_status[symbol] == 'Sell':
                if df[f'{symbol}-USD_Close'].iloc[i] <= df[f'{symbol}_EMA_50'].iloc[i] and df[f'{symbol}_SuperDir'].iloc[i] != -1: # Good to buy
                    print(f'{env.balance_amount} short', env.short_unit, env.wins, env.losses)
                    env.buy_short(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    print(f'{symbol} short', df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    env.reset_up_candlesticks([symbol])
                    env.reset_up_status([symbol])
                    env.reset_down_candlesticks([symbol])
                    env.reset_down_status([symbol])
                else: # Not ready to buy
                    env.down_candlesticks[symbol] += 1
                    if env.down_candlesticks[symbol] == 3:
                        env.reset_down_candlesticks([symbol])
                        env.reset_down_status([symbol])

            
            # Check if up_cross
            if df[f'{symbol}_EMA_50'].iloc[i-1] >= df[f'{symbol}_EMA_200'].iloc[i-1] and df[f'{symbol}_EMA_50'].iloc[i] < df[f'{symbol}_EMA_200'].iloc[i]:
                env.down_status[symbol] = 'Down_Crossed'
                print(f'{symbol} Down Crossed', df['CloseTime'].iloc[i])
            
            # Checl if up_cross and thouched
            if env.down_status[symbol] == 'Down_Crossed' and df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_EMA_50'].iloc[i]:
                env.down_status[symbol] = 'Sell'
                print(f'{symbol} short signal', df['CloseTime'].iloc[i])

            ####################################################################################################################################################

    
    else:

        if(env.short_amount == 0): 
            if df[f'{env.balance_unit}-USD_High'].iloc[i] >= env.long_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]):
                print('Win long', env.long_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.sell_long(env.long_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.wins += 1
                continue
            
            if df[f'{env.balance_unit}-USD_Low'].iloc[i] <= env.long_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]):
                print('Loss long', env.long_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.sell_long(env.long_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.losses += 1
        ###############################################################################################################################
        else:
            if df[f'{env.short_unit}-USD_Low'].iloc[i] <= env.short_price - (env.take_profit * df[f'{env.short_unit}_ATR'].iloc[i]):
                print('Win short', env.short_price - (env.take_profit * df[f'{env.short_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.sell_short(env.short_price - (env.take_profit * df[f'{env.short_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.wins += 1
                continue
            
            if df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price + (env.stop_loss * df[f'{env.short_unit}_ATR'].iloc[i]):
                print('Loss short', env.short_price + (env.stop_loss * df[f'{env.short_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.sell_short(env.short_price + (env.stop_loss * df[f'{env.short_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
                env.losses += 1
if env.balance_unit != 'USDT':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])

if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
    
print(env.balance_amount, env.balance_unit, env.wins, env.losses)

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
ETH Up Crossed 2021-06-17 03:04:59.999
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
ETH buy signal 2021-06-17 04:49:59.999
100
100
100 long USDT 0 0
ETH long 2438.0 2021-06-17 04:59:59.999
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.04099261689909762
0.040992

In [32]:
print(env.balance_amount, len(env.buys))
#plot_results(df, 'BTC', env.buys, env.sells)


93.1194385106499 101
